<a href="https://colab.research.google.com/github/Iddhie/BlockchainDataset/blob/main/exampledf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing relevant libraries and defining constants

In [ ]:
from datetime import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')
ether_api_key='FVIGP2ZZX9I44Q1NTPW9BC1R2N5D8IPZWH'
covalent_api_key='cqt_rQDyw4jwV4WyGhyhYwXY3yPKY9hB'
ether_url='https://api.etherscan.io/api'
subgraph_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Retrieving the last 1000 swaps in UniSwap using UniSwap Subgraph API

In [ ]:
query = '''
{
  swaps(first: 1000, orderBy: timestamp, orderDirection: desc) {
    timestamp
    origin
    amountUSD
    token0 {
      id
      symbol
      feesUSD
      totalValueLockedUSD
      volumeUSD
    }
    amount0
    token1 {
      id
      symbol
      feesUSD
      totalValueLockedUSD
      volumeUSD

    }
    amount1
    transaction {
      gasPrice
    }
  }
}
'''
headers = {
    'Content-Type': 'application/json',
}

# Make a POST request to the Uniswap v3 subgraph endpoint
response = requests.post(subgraph_url, json={'query': query}, headers=headers)

if response.status_code == 200:
    data = response.json()
    swaps_data = data['data']['swaps']  # Extract 'swaps' data from the response

    # Convert the data to a Pandas DataFrame
    df = pd.json_normalize(swaps_data)
    df = pd.DataFrame(df)
else:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)  # Print the error message if the request fails

# Convert 'amount0', 'amount1', and 'amountUSD' columns to numeric and round to 4 decimal places
df['amount0'] = pd.to_numeric(df['amount0'], errors='coerce').round(4)
df['amount1'] = pd.to_numeric(df['amount1'], errors='coerce').round(4)
df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce').round(4)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   timestamp                   1000 non-null   object 
 1   origin                      1000 non-null   object 
 2   amountUSD                   1000 non-null   float64
 3   amount0                     1000 non-null   float64
 4   amount1                     1000 non-null   float64
 5   token0.id                   1000 non-null   object 
 6   token0.symbol               1000 non-null   object 
 7   token0.feesUSD              1000 non-null   object 
 8   token0.totalValueLockedUSD  1000 non-null   object 
 9   token0.volumeUSD            1000 non-null   object 
 10  token1.id                   1000 non-null   object 
 11  token1.symbol               1000 non-null   object 
 12  token1.feesUSD              1000 non-null   object 
 13  token1.totalValueLockedUSD  1000 n

## Testing etherscan API to obtain historical transactions of a wallet

In [ ]:
def get_transactions(address,api_key):
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': address,
        'apikey': api_key
    }

    response = requests.get(ether_url, params=params)
    if response.status_code == 200:
        transactions = response.json().get('result', [])
        return transactions
    else:
        print(f"Failed to fetch transactions for address: {address}")
        return []

address=['0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205']
transactions = get_transactions(address,ether_api_key)
transactions=pd.DataFrame(transactions)


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   blockNumber        314 non-null    object
 1   timeStamp          314 non-null    object
 2   hash               314 non-null    object
 3   nonce              314 non-null    object
 4   blockHash          314 non-null    object
 5   transactionIndex   314 non-null    object
 6   from               314 non-null    object
 7   to                 314 non-null    object
 8   value              314 non-null    object
 9   gas                314 non-null    object
 10  gasPrice           314 non-null    object
 11  isError            314 non-null    object
 12  txreceipt_status   314 non-null    object
 13  input              314 non-null    object
 14  contractAddress    314 non-null    object
 15  cumulativeGasUsed  314 non-null    object
 16  gasUsed            314 non-null    object
 1

In [ ]:
transactions

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName
0,11446347,1607885019,0x8d9395fdd44cfa8542c0cfb4046e1f9157f3eac2a1df...,0,0xf8a14fb49814ac030fc1733f9a2e6ee51e0937680411...,135,0xa6bf93783b522eb281de2518047bfaf7bd2f1434,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,32303102740000000000,21000,55000000000,0,1,0x,,6725498,21000,7294112,0x,
1,11452957,1607973411,0xb037552b514d720394b34b4c88dd95d6a6a79847532f...,0,0x842cd4afbc7df1d7f274856c789b9693eb405a4afd6e...,21,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x00000000219ab540356cbb839cbe05303d7705fa,32000000000000000000,88983,66000000000,0,1,0x22895118000000000000000000000000000000000000...,,1525510,63875,7287502,0x22895118,"deposit(bytes pubkey, bytes withdrawal_credent..."
2,11551169,1609275122,0xbb6e3ef9fb84f9e167008dec26d6a0b8fe9271de1a5c...,1,0x0494942c95b92c6738e9f52ed427300c5cfaade31a01...,65,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,7266478826646914,174581,41000000000,0,1,0xfb3bdb41000000000000000000000000000000000000...,,6939216,133426,7189290,0xfb3bdb41,"swapETHForExactTokens(uint256 amountOut, addre..."
3,11551459,1609279279,0x3528ddda90bcd0f776c1c91ac19e26bb791a0fc71270...,2,0x6c1f7371dfd5a2595487421791d5486cf85b860185e4...,151,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x881d40237659c251811cec9c364ef91dc08d300c,138100000000000000,336194,47000000000,0,1,0x5f575529000000000000000000000000000000000000...,,11299824,174544,7189000,0x5f575529,"swap(string aggregatorId, address tokenFrom, u..."
4,11569636,1609519357,0x9f50e84215715ac9a7d41faf840b972efd4a95b0cfc1...,3,0x8b71d7e539669aeebfe15e4084b89b2c640d1aa9b60e...,174,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x88addf731df28f8bf8e71720daee0c441b9c3f73,138520225156004196,21000,65000000000,0,1,0x,,10572189,21000,7170823,0x,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,16670983,1676910971,0x6973151b13ee6f8d59e727ecd767024f21fffc5802f8...,239,0x969e7088fb72d2f57d79885178f027d7df7ea656ed70...,25,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0x3aada3e213abf8529606924d8d1c55cbdc70bf74,0,55930,45252728639,0,1,0x095ea7b3000000000000000000000000ada31f59e70a...,,2187209,46609,2069476,0x095ea7b3,"approve(address _spender, uint256 _value)"
310,16670986,1676911007,0xbb058b69678215eeb9a9a87ef086dc0b7d2f16f217aa...,240,0xb7a61af0345703a3b8afba0c90daf3ffaa62a0eda633...,31,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xada31f59e70ad18665380f21ce49d4c43f9865c2,0,177926,45515373764,0,1,0x66dfbfb4000000000000000000000000000000000000...,,3226578,142640,2069473,0x66dfbfb4,"lock(uint256 tokenId, address locker)"
311,16671055,1676911847,0xb0b2b8021bc24b41a0497423b7d877cc3d80dc152f20...,241,0xd9d608881a9d9f8618932b7b0e5ac8d86a07c5597612...,155,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0,66836,42210191218,0,1,0x095ea7b3000000000000000000000000ba1222222222...,,13191386,60311,2069404,0x095ea7b3,"approve(address _spender, uint256 _value)"
312,16671060,1676911907,0xff5d0d472fc04e49988f1601378e2c6a55bc32e12f5a...,242,0xa1cc580ccce3a1395eb2657d3e69141cdfc79d082b82...,115,0x07f5dd1fb0a005f89ac536787dd6c03ed16bd205,0xba12222222228d8ba445958a75a0704d566bf2c8,0,37254,44210750531,0,1,0x945bcec9000000000000000000000000000000000000...,,9929747,29122,2069399,0x945bcec9,"batchSwap(uint8 kind, tuple[] swaps, address[]..."


## Obtain the historical transactions of all the wallets in the dataframe using etherscan API

In [ ]:
unique_addresses = df['origin'].unique() [:635]
origin_transactions = {}  # Dictionary to store DataFrames for each origin address

for address in unique_addresses:  # Iterate over the first 5 unique origin addresses
    transactions = get_transactions(address, ether_api_key)
    transactions_df = pd.DataFrame(transactions)

    # Add a new column with the wallet address repeating for its own transaction history rows
    transactions_df['origin'] = address
    # Store the transactions_df in the dictionary with the origin address as the key
    origin_transactions[address] = transactions_df

# Concatenate the DataFrames in the dictionary into one DataFrame horizontally (row-wise)
combined_df = pd.concat(origin_transactions.values(), ignore_index=True)
combined_df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,origin
0,17033116,1681317023,0xaf7a21c70783ffece8b9bd6b88fee079c081f064e368...,44,0x0478eac946d63e1db4e933a0e04b272561be4cfd4d49...,261,0xa48634af43eec7bc0441bf34e4382a3d7c2103be,0xb68767554ce8f7694d6e81ea6fff6db52600089b,1000000000000000000,21000,...,0,1,0x,,12863649,21000,1708143,0x,,0xb68767554ce8f7694d6e81ea6fff6db52600089b
1,18665706,1701121091,0x12e0b52c7624cb2039a170e0459bc101475729154499...,0,0xe5ba3c5a43f7bfb18c92ba80a03a26764f15ec028aff...,24,0xb68767554ce8f7694d6e81ea6fff6db52600089b,0x28e261390adaa654f29dbe268109baf06e9b4cc4,0,165347,...,0,1,0xbf7408ba000000000000000000000000824a30f2984f...,,3990376,127495,75553,0xbf7408ba,,0xb68767554ce8f7694d6e81ea6fff6db52600089b
2,18665782,1701122027,0xb19e232038c9c1a085cf7263999ee69763cbccfe582f...,1,0x24f1bde78e1986a438a1453860ac3648ecdd589fdfe7...,24,0xb68767554ce8f7694d6e81ea6fff6db52600089b,0x28e261390adaa654f29dbe268109baf06e9b4cc4,0,166419,...,0,1,0xbf7408ba000000000000000000000000824a30f2984f...,,4539557,140353,75477,0xbf7408ba,,0xb68767554ce8f7694d6e81ea6fff6db52600089b
3,18665823,1701122519,0x2e188b003079d8eb57bc50272f710d863a385ae98769...,2,0x37a2ea15ff84e82740571e0e715eaafb3deb11f17070...,57,0xb68767554ce8f7694d6e81ea6fff6db52600089b,0x28e261390adaa654f29dbe268109baf06e9b4cc4,0,168016,...,0,1,0xbf7408ba000000000000000000000000824a30f2984f...,,6034473,110361,75436,0xbf7408ba,,0xb68767554ce8f7694d6e81ea6fff6db52600089b
4,18665903,1701123491,0x4d5dd90ee10a94b8fc737e9afb6e39abf2cd4c1968f5...,3,0x2227371177fb081e78417bb248267abe05b14465fb3b...,183,0xb68767554ce8f7694d6e81ea6fff6db52600089b,0x28e261390adaa654f29dbe268109baf06e9b4cc4,0,169688,...,0,1,0x493a288b000000000000000000000000824a30f2984f...,,16516095,140086,75356,0x493a288b,,0xb68767554ce8f7694d6e81ea6fff6db52600089b


In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161722 entries, 0 to 1161721
Data columns (total 21 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   blockNumber        1161722 non-null  object
 1   timeStamp          1161722 non-null  object
 2   hash               1161722 non-null  object
 3   nonce              1161722 non-null  object
 4   blockHash          1161722 non-null  object
 5   transactionIndex   1161722 non-null  object
 6   from               1161722 non-null  object
 7   to                 1161722 non-null  object
 8   value              1161722 non-null  object
 9   gas                1161722 non-null  object
 10  gasPrice           1161722 non-null  object
 11  isError            1161722 non-null  object
 12  txreceipt_status   1161722 non-null  object
 13  input              1161722 non-null  object
 14  contractAddress    1161722 non-null  object
 15  cumulativeGasUsed  1161722 non-null  object
 16  

## Saving the dataset to Google Drive

In [ ]:
combined_df.to_csv('/content/drive/My Drive/sampleData.csv', index=True)

## Dataset to send

In [ ]:
unique_addresses = df['origin'].unique()[:10]
origin_transactions = {}  # Dictionary to store DataFrames for each origin address

for address in unique_addresses:  # Iterate over the first 5 unique origin addresses
    transactions = get_transactions(address, ether_api_key)
    transactions_df = pd.DataFrame(transactions)

    # Add a new column with the wallet address repeating for its own transaction history rows
    transactions_df['origin'] = address

    # Store the transactions_df in the dictionary with the origin address as the key
    origin_transactions[address] = transactions_df

# Concatenate the DataFrames in the dictionary into one DataFrame horizontally (row-wise)
sample_df = pd.concat(origin_transactions.values(), ignore_index=True)

sample_df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,origin
0,18611384,1700463719,0xa127ef845a682f48654bb9b67ac1565c9560ceb9f865...,203,0xb0b7085018acd1d531c9aa6b4e2a756b3627e5f08edd...,139,0x465a646e92746a7191ec6ee7223d3b78d20367e5,0xf25087d955b8509da9f0195cc1bd21b73890316d,35000000000000000,21000,...,0,1,0x,,10896528,21000,130361,0x,,0xf25087d955b8509da9f0195cc1bd21b73890316d
1,18611399,1700463899,0x2193d9d99f2e23851d84a30ea19d445ed8c856bae80e...,0,0xfb3c267b0b1242b15d49e658f674107eed380890b49f...,250,0xf25087d955b8509da9f0195cc1bd21b73890316d,0xdac17f958d2ee523a2206206994597c13d831ec7,0,48501,...,0,1,0x095ea7b3000000000000000000000000000000000022...,,21818901,48501,130346,0x095ea7b3,"approve(address _spender, uint256 _value)",0xf25087d955b8509da9f0195cc1bd21b73890316d
2,18611408,1700464007,0x10ef3ccd6005ff692adc86092c32f6b9c8e84783def7...,1,0x427dde8ed06c33b7d377bbff9eb0f8355e69094cc1c7...,82,0xf25087d955b8509da9f0195cc1bd21b73890316d,0xf3f04555f8fda510bfc77820fd6eb8446f59e72d,100000000000000000,129265,...,0,1,0x1eb1cff0000000000000000000000000000000000000...,,5627443,129263,130337,0x1eb1cff0,,0xf25087d955b8509da9f0195cc1bd21b73890316d
3,18611467,1700464727,0x007d2b26f4cbabde5d6fbff382876a4e7ffcf784f626...,2,0xd561eabddc79cbc4eee60e7e795261583a6ff8ae878a...,115,0xf25087d955b8509da9f0195cc1bd21b73890316d,0x26ba41f9a94392901ffd8d05c76a76c8b786fd36,100000000000000000,21000,...,0,1,0x,,6820494,21000,130278,0x,,0xf25087d955b8509da9f0195cc1bd21b73890316d
4,18611498,1700465099,0xa6ff1c462869a6fd17dffbd2fde06fd45098e8d79335...,3,0x48acab488e8ccf7694429d9518e32e44c26f4feef251...,31,0xf25087d955b8509da9f0195cc1bd21b73890316d,0x26ba41f9a94392901ffd8d05c76a76c8b786fd36,950000000000000000,21000,...,0,1,0x,,4994011,21000,130247,0x,,0xf25087d955b8509da9f0195cc1bd21b73890316d


In [ ]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15103 entries, 0 to 15102
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   blockNumber        15103 non-null  object
 1   timeStamp          15103 non-null  object
 2   hash               15103 non-null  object
 3   nonce              15103 non-null  object
 4   blockHash          15103 non-null  object
 5   transactionIndex   15103 non-null  object
 6   from               15103 non-null  object
 7   to                 15103 non-null  object
 8   value              15103 non-null  object
 9   gas                15103 non-null  object
 10  gasPrice           15103 non-null  object
 11  isError            15103 non-null  object
 12  txreceipt_status   15103 non-null  object
 13  input              15103 non-null  object
 14  contractAddress    15103 non-null  object
 15  cumulativeGasUsed  15103 non-null  object
 16  gasUsed            15103 non-null  objec

In [ ]:
sample_df.to_csv('/content/drive/My Drive/sample_dataframe2.csv', index=True)

## Taking Token Balances of the Wallet Addresses Using Covalent

In [ ]:
# Replace 'your_wallet_address' with the specific wallet address you want to query
wallet_address = '0xb68767554ce8f7694d6e81ea6fff6db52600089b'

network = 1

# Construct the API endpoint for getting token balances
endpoint = f'https://api.covalenthq.com/v1/{network}/address/{wallet_address}/balances_v2/'

# Set up the request headers
headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {covalent_api_key}'}

# Make the API request
response = requests.get(endpoint, headers=headers)

# Process the response
if response.status_code == 200:
    # Parse and extract token balances from the response
    token_balances = response.json()['data']['items']
    # Process token balances as needed
    print(f'Token balances for {wallet_address}: {token_balances}')
else:
    print(f'Error fetching data for {wallet_address}. Status code: {response.status_code}')

Token balances for 0xb68767554ce8f7694d6e81ea6fff6db52600089b: [{'contract_decimals': 18, 'contract_name': 'Ether', 'contract_ticker_symbol': 'ETH', 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'supports_erc': None, 'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png', 'contract_display_name': 'Ether', 'logo_urls': {'token_logo_url': 'https://logos.covalenthq.com/tokens/1/0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee.png', 'protocol_logo_url': None, 'chain_logo_url': 'https://www.datocms-assets.com/86369/1669653891-eth.svg'}, 'last_transferred_at': '2023-12-08T09:29:11Z', 'native_token': True, 'type': 'cryptocurrency', 'is_spam': False, 'balance': '2084948171924366040', 'balance_24h': '2133190206424845455', 'quote_rate': 2366.755, 'quote_rate_24h': 2370.673, 'quote': 4934.561, 'pretty_quote': '$4,934.56', 'quote_24h': 5057.0967, 'pretty_quote_24h': '$5,057.10', 'protocol_metadata': None, 'nft_data': None}]


In [ ]:
# List of wallet addresses to query
wallet_addresses = [
    '0x8eb2283f696f2a130134d46e28d3528e19e16868',
    '0xb68767554ce8f7694d6e81ea6fff6db52600089b',
    '0x88addf731df28f8bf8e71720daee0c441b9c3f73',
]

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['Wallet Address', 'Token Balances'])

for wallet_address in wallet_addresses:
    # Construct the API endpoint for getting token balances
    endpoint = f'https://api.covalenthq.com/v1/{network}/address/{wallet_address}/balances_v2/'

    # Set up the request headers
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {covalent_api_key}'}

    # Make the API request
    response = requests.get(endpoint, headers=headers)

    # Process the response
    if response.status_code == 200:
        # Parse and extract token balances from the response
        token_balances = response.json()['data']['items']

        # Append data to the DataFrame
        result_df = result_df.append({
            'Wallet Address': wallet_address,
            'Token Balances': token_balances
        }, ignore_index=True)
    else:
        print(f'Error fetching data for {wallet_address}. Status code: {response.status_code}')

# Display the resulting DataFrame
print(result_df)

<ipython-input-91-b0d22c41b12f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-91-b0d22c41b12f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({


                               Wallet Address  \
0  0x8eb2283f696f2a130134d46e28d3528e19e16868   
1  0xb68767554ce8f7694d6e81ea6fff6db52600089b   
2  0x88addf731df28f8bf8e71720daee0c441b9c3f73   

                                      Token Balances  
0  [{'contract_decimals': 6, 'contract_name': 'Te...  
1  [{'contract_decimals': 18, 'contract_name': 'E...  
2  [{'contract_decimals': 18, 'contract_name': 'E...  


<ipython-input-91-b0d22c41b12f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
